# 0 Overview
This notebook is a supplement to our introductory notebook, `lecture 3 standard incomplete markets steady state slides.ipynb`, for more advanced users.

Here, we'll *profile* the basic standard incomplete markets code we introduced in that notebook, to see where there are opportunities to speed it up. Then, we'll implement some key improvements. By the time we finish, the computation of a steady state will be approximately twice as fast as in the introductory notebook, and fully on par with the state of the art: 17 milliseconds to calculate the policy function and distribution (each on a state space of dimension 3500) on my run-of-the-mill Macbook Air laptop.

The faster version of `sim_steady_state.py`, which uses the functions from this notebook, will be in `sim_steady_state_fast.py`, which is also an excellent place to look if you want to briefly see the changes that have been made.

This notebook has 5 sections:

**[Section 1: introduction to profiling](#section1)**. We'll discuss what it means to *profile* code, to see how long each chunk of code requires to run, and which parts of the code appear to be the major bottlenecks. As an example, we'll start to profile the major functions from our last notebook, which are included in `sim_steady_state.py` (imported as `sim`).

**[Section 2: profiling and improving `distribution_ss`](#section2).** We'll profile `distribution_ss`, which solves for the steady-state distribution given a policy function, in detail. Using our insights from profiling, we'll make several specific improvements, which together will make `distribution_ss` approximately twice as fast.

**[Section 3: profiling and improving `policy_ss`](#section3).** We'll profile `policy_ss`, which solves for the steady-state policy function given parameters. Combining information from profiling with insights from section 2, we'll speed up `policy_ss` by a factor of more than two.

**[Section 4: overall steady-state performance](#section4).** We'll combine the improved versions of `distribution_ss` and `policy_ss` to obtain an updated `steady_state` function, which doubles the speed of our old one, calculating the steady-state policy and distribution in about 50 ms on an old laptop.

**[Section 5 (addendum): replacing `get_lottery`](#section5).** Here we make one extra change, which will be useful later on. In section 2, we see that the `get_lottery` function from our previous notebook, which converts the continuous policy function into a lottery over adjacent gridpoints, is not very fast—but that this doesn't matter for computing the steady state, since then it only needs to be done once. Here, we use the improved interpolation code from section 3 to speed it up anyway, which will be invaluable as we move to dynamics.

*Note*: although one important way to improve code performance is through [parallelization](https://en.wikipedia.org/wiki/Parallel_computing), we won't cover that in this notebook: our code will not be explicitly parallel in any way, though very small parts of it (e.g. matrix multiplication) may be automatically parallelized by NumPy. Parallelization could likely achieve significant additional gains.

**Preliminaries.** Before we start, we'll import the basic Python libraries we'll be using:

In [1]:
import numpy as np
import numba
from scipy import linalg

and also the code developed in our previous notebook, along with the example calibration that is included with it:

In [2]:
import sim_steady_state as sim
calibration = sim.example_calibration()

You may recall that this calibration has a state space of dimension 3500: there are 500 gridpoints for the grid of assets, and 7 discrete income states.

In [3]:
len(calibration['a_grid']), len(calibration['y'])

(500, 7)

<a id="section1"></a>
# 1 Introduction to profiling
Broadly, to "profile" code is to analyze code to see what parts of the code take the most time, memory, and so on to run, and also which parts of the code are run the most.

In this notebook, we'll be using the profiling tools that are available in Jupyter and its sibling IPython. (These tools can also be called from the Python language itself, though we will not do this.) We'll also focus on speed, rather than memory footprint. 

# 1.1 `%time` and `%timeit`
The most basic profiling tools available in Jupyter/IPython are the "magic" commands `%time` and `%timeit`.

The first of these, `%time`, reports the amount of time that it takes to run the line of code once. There are two measures, and we will focus on the second, "wall time", which is literally the amount of time that elapsed while running the line of code.

(Technical notes: this can sometimes be too high if your computer gets distracted and also works hard on something else while running the line of code, in which case you might notice a big discrepancy with the other measure, "CPU time". "CPU time", on the other hand, can be higher if your code is parallelized, since it counts each core separately.)

Let's run `%time` on our steady state function. The dict `calibration` contains exactly the inputs needed by that function, and feeding it in as an input with two stars, `**calibration`, tells Python to "unpack" the dict and feed its key-value pairs as arguments to the function.

In [4]:
%time ss = sim.steady_state(**calibration)

CPU times: user 201 ms, sys: 18.9 ms, total: 220 ms
Wall time: 252 ms


That took a while, with wall time of more than a quarter of a second! Let's run it again:

In [5]:
%time ss = sim.steady_state(**calibration)

CPU times: user 33 ms, sys: 853 µs, total: 33.9 ms
Wall time: 33.3 ms


Now this is much quicker, with wall time for finding the steady state equal to only 1/30th of a second!

This discrepancy is because our steady-state code contains a function, `forward_policy`, that is accelerated with [Numba](https://numba.pydata.org/). The first time that function is run, Numba needs to *compile* it, which requires a one-time delay, analogous to (but usually far quicker than) the up-front time needed for compilation in a language like Fortran or C.

Usually, when profiling code, we want to ignore this compilation time: it's a one-time cost, and will quickly become irrelevant in comparison to speed improvements when we run our code enough times. (And if it doesn't become irrelevant, perhaps because the code is already fast enough, or we tend not to run it very many times in a session, then optimization with Numba was probably unnecessary in the first place.)

Hence, we have learned our first lesson: *to make sure that our profiling is unaffected by Numba compilation time, run the code once before timing it whenever compilation might be an issue*. We'll call this a **burn-in** run.

The other command, `%timeit`, is similar to `%time`, but runs a line of code many times to get a better estimate of its running time, and also does some other more sophisticated things to try and get as accurate an estimate as possible. It usually produces a lower and more accurate estimate than `%time`, but takes longer (and another disadvantage is that you can't use it to save anything to a variable). Let's try it here:

In [6]:
%timeit sim.steady_state(**calibration)

31.8 ms ± 39.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


This 32 ms is only slightly lower than the 33 ms from `%time` above, so in this case there is little difference. The extra accuracy provided by `%timeit` tends to be more important when we are looking at functions that are very fast, with runtimes that are at most a few milliseconds, and maybe in the microseconds or nanoseconds.

If we look at its source code in `sim_steady_state.py`, the two key calls made by the `steady_state` function are, first, to `policy_ss` to get the steady-state marginal value function and policy functions:

In [7]:
%time Va, a, c = sim.policy_ss(**calibration)

CPU times: user 23.8 ms, sys: 673 µs, total: 24.5 ms
Wall time: 23.9 ms


and then to `distribution_ss` to get the steady-state distribution:

In [8]:
%time D = sim.distribution_ss(calibration['Pi'], a, calibration['a_grid'])

CPU times: user 9.81 ms, sys: 1.07 ms, total: 10.9 ms
Wall time: 10 ms


We see that `policy_ss` is the more costly function here, but that `distribution_ss` also takes up a substantial amount of time.

To delve further, we want to look and see what parts of the functions `policy_ss` and `distribution_ss` are most costly. We could continue to look at the source and run `%time` or `%timeit` on each line by hand, but there is a much more convenient way: using a *line profiling* tool.

# 1.2 Line profiling
Beyond `%time` and `%timeit`, the easiest-to-understand profiling tool available for Python is the *line profiler*.

Unfortunately, this requires a separate installation. At the command line, you can install it with:
```
python -m pip install line_profiler
```
and then load it within a Jupyter or IPython session with the line
```
%load_ext line_profiler
```
So that this notebook is still usable even if you haven't installed the line profiler, I have commented out the lines that use it, and copy-pasted the results directly into the notebook. By uncommenting the lines, you can run the line profiler yourself. 

In [9]:
# uncomment below to load line profiler extension
%load_ext line_profiler

As an initial example, let's replicate what we already did manually with `%time`: looking at our `steady_state` function and seeing how long each step takes.

To do so, we run the command `%lprun -f `, followed by, first, the function within which we want to see lines profiled (here, `sim.steady_state`), and second, the line of code we want to run (here, `sim.steady_state(**calibration)`).

(To make the results readable, either zoom out in your browser or widen your browser window as much as possible!)

In [10]:
# uncomment below to run line profiler yourself, if you've loaded extension with %load_ext line_profiler
# %lprun -f sim.steady_state sim.steady_state(**calibration)

```
Timer unit: 1e-09 s

Total time: 0.052659 s
File: /Users/matthewrognlie/Dropbox/Courses/Northwestern grad/Econ 411-3 Macro 2024/sim_steady_state.py
Function: steady_state at line 173

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   173                                           def steady_state(Pi, a_grid, y, r, beta, eis):
   174         1   40782000.0    4e+07     77.4      Va, a, c = policy_ss(Pi, a_grid, y, r, beta, eis)
   175         1     266000.0 266000.0      0.5      a_i, a_pi = get_lottery(a, a_grid)
   176         1   11598000.0    1e+07     22.0      D = distribution_ss(Pi, a, a_grid)
   177                                               
   178         2       4000.0   2000.0      0.0      return dict(D=D, Va=Va, 
   179         1          0.0      0.0      0.0                  a=a, c=c, a_i=a_i, a_pi=a_pi,
   180         1       9000.0   9000.0      0.0                  A=np.vdot(a, D), C=np.vdot(c, D),
   181         1          0.0      0.0      0.0                  Pi=Pi, a_grid=a_grid, y=y, r=r, beta=beta, eis=eis)
```

This delivers a similar message to what we saw earlier: the `policy_ss` and `distribution_ss` functions take up essentially all the time in this function, with the split being approximately 3/4 for `policy_ss` and 1/4 for `distribution_ss`.

The other information—how many times each line is run ("hits"), and then distinguishing between total time and time per hit—is not yet useful, since both `policy_ss` and `distribution_ss` are only run once.

(Note that there is some overhead here, and the line profiler overstates the actual amount of time the function takes to run.)

<a id="section2"></a>
# 2 Profiling and improving `distribution_ss`
In the last section, we saw that about 75% of the time to find a steady state was in the `policy_ss` function that iterates backward to find the steady-state policy function, and 25% in the `distribution_ss` function that iterates forward to find the steady-state distribution.

Although `distribution_ss` is the lesser of the two contributors to running time here, we'll start by speeding it up—both because it's a simpler warm-up exercise, and because it will teach us some useful lessons that will carry over to `policy_ss`.

## 2.1 Profiling `distribution_ss`
Let's run the line profiler on `distribution_ss`. For simplicity, we'll run the same command to obtain the whole steady state, `sim.steady_state(**calibration)`, but then ask the profiler to only look at the `distribution_ss` function:

In [11]:
# uncomment below to run line profiler yourself, if you've loaded extension with %load_ext line_profiler
# %lprun -f sim.distribution_ss sim.steady_state(**calibration)

```
Timer unit: 1e-09 s

Total time: 0.011969 s
File: /Users/matthewrognlie/Dropbox/Courses/Northwestern grad/Econ 411-3 Macro 2024/sim_steady_state.py
Function: distribution_ss at line 156

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   156                                           def distribution_ss(Pi, a, a_grid, tol=1E-10):
   157         1     246000.0 246000.0      2.1      a_i, a_pi = get_lottery(a, a_grid)
   158                                               
   159                                               # as initial D, use stationary distribution for s, plus uniform over a
   160         1    2167000.0    2e+06     18.1      pi = stationary_markov(Pi)
   161         1      20000.0  20000.0      0.2      D = pi[:, np.newaxis] * np.ones_like(a_grid) / len(a_grid)
   162                                               
   163                                               # now iterate until convergence to acceptable threshold
   164       581      91000.0    156.6      0.8      for _ in range(10_000):
   165       581    6303000.0  10848.5     52.7          D_new = forward_iteration(D, Pi, a_i, a_pi)
   166       581    3034000.0   5222.0     25.3          if np.max(np.abs(D_new - D)) < tol:
   167         1          0.0      0.0      0.0              return D_new
   168       580     108000.0    186.2      0.9          D = D_new
```

Note that the "timer unit" of 1e-09 s means that the line times in this report are in *nanoseconds*, i.e. billionths of a second.

We can draw 4 lessons from the above:

1. The single biggest cost, as expected, is running the actual forward iteration on the distribution, line 165, 581 times. This takes 50% of runtime.


2. But there is a shockingly high cost from line 166, which simply *checks whether the convergence threshold has been reached*. At 25% of runtime, this is half the cost of the actual forward iteration!


3. There is also a pretty high cost from line 160, which runs `stationary_markov` to obtain the stationary distribution for the exogenous Markov process itself, which is then used to construct a nicer initial guess for the distribution.  This line wasn't strictly necessary, and we included it to reduce the number of iterations a little bit—but at 18% of runtime, it's probably not worth it.


4. Line 156, which runs `get_lottery` to find the "lottery" representation of the policy function, is insignificant here, at only 2% of runtime for the steady state. But that's because we only need to run it once, on the steady-state policy function, rather than separately for each iteration. Per run, it actually has a much higher cost than our forward iteration function (246 vs. 11 microseconds). In a dynamic context, where the policy function is changing each period and we'd need to rerun this line, it would become a major part of the cost.


Let's follow up on point 3, and both time and profile the `stationary_markov` function.

In [12]:
%timeit sim.stationary_markov(calibration['Pi'])

1.25 ms ± 9.96 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
# uncomment below to run line profiler yourself, if you've loaded extension with %load_ext line_profiler
# %lprun -f sim.stationary_markov sim.steady_state(**calibration)

```
Timer unit: 1e-09 s

Total time: 0.002397 s
File: /Users/matthewrognlie/Dropbox/Courses/Northwestern grad/Econ 411-3 Macro 2024/sim_steady_state.py
Function: stationary_markov at line 51

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    51                                           def stationary_markov(Pi, tol=1E-14):
    52                                               # start with uniform distribution over all states
    53         1       2000.0   2000.0      0.1      n = Pi.shape[0]
    54         1       9000.0   9000.0      0.4      pi = np.full(n, 1/n)
    55                                               
    56                                               # update distribution using Pi until successive iterations differ by less than tol
    57       556      89000.0    160.1      3.7      for _ in range(10_000):
    58       556     510000.0    917.3     21.3          pi_new = Pi.T @ pi
    59       556    1687000.0   3034.2     70.4          if np.max(np.abs(pi_new - pi)) < tol:
    60         1          0.0      0.0      0.0              return pi_new
    61       555     100000.0    180.2      4.2          pi = pi_new
```

This is an alarming echo of point 2 from above: the vast majority of the time is spent on line 59, which simply checks whether or not there has been convergence!

## 2.2 Speeding up `stationary_markov`
First, let's improve `stationary_markov`, directly addressing point 3 from above, and in the process learning how to address point 2.

The key is to avoid all the time spent in line 59, which checks whether there has been convergence. The simplest way to do this is to simply *cut down the number of times* this test is run. Given that it takes 556 iterations to achieve convergence, testing on every single iteration seems excessive. Why not test every 10 iterations? We'll end up doing a few more iterations than needed, but vastly cut down on the test time, which is the dominant cost here.

Below we rewrite the function to achieve this, only performing the comparison when the iteration counter `it` is a multiple of 10 (i.e. when `it % 10 == 0`, where `%` is the modulo function). Here we're making use of the "short-circuit" feature of Python `and`: if the first argument evaluates as false, the second one isn't even calculated.

Also note that we're using `%%writefile` to write this cell to a file, since the line profiler works better when a function is from a file.

In [14]:
# %%writefile temp1.py
# uncomment above to write file if you're using line profiler yourself
import numpy as np
def stationary_markov(Pi, tol=1E-14):
    # start with uniform distribution over all states
    n = Pi.shape[0]
    pi = np.full(n, 1/n)
    
    # update distribution using Pi until successive iterations differ by less than tol
    for it in range(10_000):
        pi_new = Pi.T @ pi
        if it % 10 == 0 and np.max(np.abs(pi_new - pi)) < tol:
            return pi_new
        pi = pi_new

Now let's profile this new function.

In [15]:
# uncomment below to run line profiler yourself, if you've loaded extension with %load_ext line_profiler
# import temp1
# temp1.stationary_markov(calibration['Pi']) # burn-in new module
# %lprun -f temp1.stationary_markov temp1.stationary_markov(calibration['Pi'])

```
Timer unit: 1e-09 s

Total time: 0.000888 s
File: /Users/matthewrognlie/Dropbox/Courses/Northwestern grad/Econ 411-3 Macro 2024/temp1.py
Function: stationary_markov at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     3                                           def stationary_markov(Pi, tol=1E-14):
     4                                               # start with uniform distribution over all states
     5         1       2000.0   2000.0      0.2      n = Pi.shape[0]
     6         1      27000.0  27000.0      3.0      pi = np.full(n, 1/n)
     7                                               
     8                                               # update distribution using Pi until successive iterations differ by less than tol
     9       561      69000.0    123.0      7.8      for it in range(10_000):
    10       561     434000.0    773.6     48.9          pi_new = Pi.T @ pi
    11       561     282000.0    502.7     31.8          if it % 10 == 0 and np.max(np.abs(pi_new - pi)) < tol:
    12         1          0.0      0.0      0.0              return pi_new
    13       560      74000.0    132.1      8.3          pi = pi_new
```

Now the total time has fallen substantially, and the line that checks for convergence accounts for a smaller share of total runtime.

Indeed, if we use `%timeit` to get a very accurate measurement of the total time needed to run this function, we see that it's already very low, just 430 µs—a factor-of-3 improvement from our earlier function!

In [16]:
%timeit stationary_markov(calibration['Pi'])

431 µs ± 294 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


**Making the convergence test even cheaper.** This is already good enough for most practical purposes, but we can make one nice additional improvement.

Using `np.max(np.abs(pi_new - pi)) < tol` to check whether the arrays are equal to within a tolerance is highly inefficient: this command first constructs two intermediate arrays (`pi_new - pi`, and then its absolute value) and then takes a maximum before comparing to the tolerance. Most of the time, it will suffice simply to compare one or two entries of `pi_new` and `pi` and see that we're not yet within the tolerance. It's also always good to avoid creation of intermediate arrays in memory, which is costly (and a much bigger deal for larger arrays). We can write a very simple Numba-accelerated function called `equal_tolerance` that does this:

In [17]:
@numba.njit
def equal_tolerance(x1, x2, tol):
    # "ravel" flattens both x1 and x2, without making copies, so we can compare the
    # with a single for loop even if they have multiple dimensions. not needed for now,
    # but will be useful later!
    x1 = x1.ravel()
    x2 = x2.ravel()
    for i in range(len(x1)):
        if np.abs(x1[i] - x2[i]) >= tol:
            return False
    return True

and call it in the new `stationary_markov` function:

In [18]:
def stationary_markov(Pi, tol=1E-14):
    # start with uniform distribution over all states
    n = Pi.shape[0]
    pi = np.full(n, 1/n)
    
    # update distribution using Pi until successive iterations differ by less than tol
    for it in range(10_000):
        pi_new = Pi.T @ pi
        if it % 10 == 0 and equal_tolerance(pi, pi_new, tol):
            return pi_new
        pi = pi_new

In [19]:
stationary_markov(calibration['Pi']) # burn-in
%timeit stationary_markov(calibration['Pi'])

342 µs ± 638 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


This change has brought the runtime down from 1.24 ms to 340 µs, a decrease of over 70%. At this point, `stationary_markov` is certainly no longer a major bottleneck.

**Alternative: dominant eigenvector.** One can achieve even more impressive speeds using SciPy's built-in eigenvalue and eigenvector solver to look for the dominant left eigenvector of `Pi`. We'll use `%%timeit`, a generalization of `%timeit` that times an entire cell, to time this:

In [20]:
%%timeit
w, v = linalg.eig(calibration['Pi'].T) # to get left eigenvectors, take transpose of Pi
pi = v[:, np.argmax(w)].real  # take eigenvector for maximum eigenvalue, ignore imaginary part (numerical error)

14.6 µs ± 57.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


This delivers almost another 20-fold improvement—but from a very low base already. We won't use this approach, because it doesn't generalize as well when `Pi` is not a simple matrix anymore (and instead is sparse, or a product of several independent matrices, etc). It is also possible to obtain comparable speed gains by compiling `stationary_markov` with Numba.

## 2.3 Speeding up `distribution_ss`
With the insights from working on `stationary_markov`, we are now ready to improve `distribution_ss`. Right now, a precise timing says that the original function takes 9 ms:

In [21]:
%timeit sim.distribution_ss(calibration['Pi'], a, calibration['a_grid'])

8.75 ms ± 8.23 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Let's rewrite `distribution_ss` to use the new `stationary_markov`, to only test for convergence every 10th iteration, and also to use `equal_tolerance` for those tests, just like in the previous section:

In [22]:
def distribution_ss(Pi, a, a_grid, tol=1E-10):
    a_i, a_pi = sim.get_lottery(a, a_grid)
    
    # as initial D, use stationary distribution for s, plus uniform over a
    pi = stationary_markov(Pi)
    D = pi[:, np.newaxis] * np.ones_like(a_grid) / len(a_grid)
    
    # now iterate until convergence to acceptable threshold
    for it in range(10_000):
        D_new = sim.forward_iteration(D, Pi, a_i, a_pi)
        if it % 10 == 0 and equal_tolerance(D_new, D, tol):
            return D_new
        D = D_new

How much of an improvement did we get from these? A one-third reduction in runtime!

In [23]:
distribution_ss(calibration['Pi'], a, calibration['a_grid']) # burn-in
%timeit distribution_ss(calibration['Pi'], a, calibration['a_grid'])

5.81 ms ± 37.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


<a id="section3"></a>
# 3 Profiling and improving `policy_ss`
Now we move to the more major project: more quickly obtaining the steady-state policy function in `policy_ss`. Start by running the line profiler:

In [24]:
# uncomment below to run line profiler yourself, if you've loaded extension with %load_ext line_profiler
# %lprun -f sim.policy_ss sim.steady_state(**calibration)

```
Timer unit: 1e-09 s

Total time: 0.030549 s
File: /Users/matthewrognlie/Dropbox/Courses/Northwestern grad/Econ 411-3 Macro 2024/sim_steady_state.py
Function: policy_ss at line 108

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   108                                           def policy_ss(Pi, a_grid, y, r, beta, eis, tol=1E-9):
   109                                               # initial guess for Va: assume consumption 5% of cash-on-hand, then get Va from envelope condition
   110         1      36000.0  36000.0      0.1      coh = y[:, np.newaxis] + (1+r)*a_grid
   111         1       4000.0   4000.0      0.0      c = 0.05 * coh
   112         1       7000.0   7000.0      0.0      Va = (1+r) * c**(-1/eis)
   113                                               
   114                                               # iterate until maximum distance between two iterations falls below tol, fail-safe max of 10,000 iterations
   115       541      82000.0    151.6      0.3      for it in range(10_000):
   116       541   27248000.0  50366.0     89.2          Va, a, c = backward_iteration(Va, Pi, a_grid, y, r, beta, eis)
   117                                                   
   118                                                   # after iteration 0, can compare new policy function to old one
   119       541    3091000.0   5713.5     10.1          if it > 0 and np.max(np.abs(a - a_old)) < tol:
   120         1          0.0      0.0      0.0              return Va, a, c
   121                                                   
   122       540      81000.0    150.0      0.3          a_old = a
```

The message here is quite clear: a small amount of the total runtime (10%) is in the convergence check, which we already know how to improve, and essentially all the rest is in the `backward_iteration` function

Let's turn to that function, and run the line profiler, next:

In [25]:
# uncomment below to run line profiler yourself, if you've loaded extension with %load_ext line_profiler
# %lprun -f sim.backward_iteration sim.steady_state(**calibration)

```
Timer unit: 1e-09 s

Total time: 0.027014 s
File: /Users/matthewrognlie/Dropbox/Courses/Northwestern grad/Econ 411-3 Macro 2024/sim_steady_state.py
Function: backward_iteration at line 86

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    86                                           def backward_iteration(Va, Pi, a_grid, y, r, beta, eis):
    87                                               # step 1: discounting and expectations
    88       541    2900000.0   5360.4     10.7      Wa = beta * Pi @ Va
    89                                               
    90                                               # step 2: solving for asset policy using the first-order condition
    91       541     655000.0   1210.7      2.4      c_endog = Wa**(-eis)
    92       541    1976000.0   3652.5      7.3      coh = y[:, np.newaxis] + (1+r)*a_grid
    93                                               
    94       541     287000.0    530.5      1.1      a = np.empty_like(coh)
    95      4328     581000.0    134.2      2.2      for e in range(len(y)):
    96      3787   17611000.0   4650.4     65.2          a[e, :] = np.interp(coh[e, :], c_endog[e, :] + a_grid, a_grid)
    97                                                   
    98                                               # step 3: enforcing the borrowing constraint and backing out consumption
    99       541    1073000.0   1983.4      4.0      a = np.maximum(a, a_grid[0])
   100       541     624000.0   1153.4      2.3      c = coh - a
   101                                               
   102                                               # step 4: using the envelope condition to recover the derivative of the value function
   103       541    1227000.0   2268.0      4.5      Va = (1+r) * c**(-1/eis)
   104                                               
   105       541      80000.0    147.9      0.3      return Va, a, c
```

By far the highest cost here is in the linear interpolation in line 96. Speedup of this is clearly the highest priority. Note that a separate call to `np.interp` is made for every exogenous state `e`, which may be part of the cost here (also, using `for` to loop over a line in pure Python tends to be quite costly in general).

One additional useful observation is that the points in the array `coh[e, :]` are increasing. `np.interp`, which only requires that the data points `c_endog[e, :]` are increasing, does not know to take advantage of this. It's a useful fact, because it means we don't need to search the entire array `c_endog[e, :]` each time to locate a point in `coh[e, :]`; instead, we can just look to the right of where we found the last point.

Let's write a Numba-accelerated function that uses this insight to do interpolation, as an alternative version of `np.interp`. It's a bit subtle, so we won't dwell on exactly how it works (which, unlike the other points we've discussed, does not have the same broad applicability) except in the comments.

In [26]:
@numba.njit
def interpolate_monotonic(x, xp, yp):
    """Linearly interpolate the data points (xp, yp) and evaluate at x"""
    nx, nxp = x.shape[0], xp.shape[0]
    y = np.empty(nx)
    
    # at any given moment, we are looking between data points with indices xp_i and xp_i+1
    # we'll keep track of xp_i and also the values of xp at xp_i and xp_i+1
    # (note: if x is outside range of xp, we'll use closest data points and extrapolate)
    xp_i = 0
    xp_lo = xp[xp_i]
    xp_hi = xp[xp_i + 1]
    
    # iterate through all points in x
    for xi_cur in range(nx):
        x_cur = x[xi_cur]
        while xp_i < nxp - 2:
            # if current x (x_cur) is below upper data point (xp_hi), we're good
            if x_cur < xp_hi:
                break
                
            # otherwise, we need to look at the next pair of data points until x_cur is less than xp_hi
            # (so between xp_lo and xp_hi)
            xp_i += 1
            xp_lo = xp_hi
            xp_hi = xp[xp_i + 1]

        # find the pi such that x_cur = pi*x_lo + (1-pi)*x_hi
        pi = (xp_hi - x_cur) / (xp_hi - xp_lo)
        
        # use this pi to interpolate the y
        y[xi_cur] = pi * yp[xp_i] + (1 - pi) * yp[xp_i + 1]
    return y

Let's quickly check that on sample data, this gives the same answer as `np.interp` up to numerical error. (Important side note: this actually will differ from `np.interp` if some `x` are outside the range of the `xp`, since it will linearly extrapolate, whereas `np.interp` just uses the nearest point. Since we prefer the extrapolation—although it should generally not be necesasry—this is actually desirable.)

In [27]:
x = np.linspace(1, 3, 20)
xp = np.linspace(0, 4, 20)
yp = np.random.rand(20)
np.max(np.abs(interpolate_monotonic(x, xp, yp) - np.interp(x, xp, yp)))

1.1102230246251565e-16

Since the `for` loop in pure Python is also costly, let's write a wrapper around this that iterates over the outer dimension (except for the third argument, where we don't need that):

In [28]:
@numba.njit
def interpolate_monotonic_loop(x, xp, yp):
    ne = x.shape[0]
    y = np.empty_like(x)
    for e in range(ne):
        y[e, :] = interpolate_monotonic(x[e, :], xp[e, :], yp)
    return y

Let's gauge the overall speed improvement on data with the same dimensions as ours, where we'll make up the `c_endog`.

In [29]:
a_grid = calibration['a_grid']
y = calibration['y']
coh = y[:, np.newaxis] + (1+calibration['r'])*a_grid
c_endog = 0.05 * coh

How long does this interpolation take using the old method?

In [30]:
%%timeit
a = np.empty_like(coh)
for e in range(len(y)):
    a[e, :] = np.interp(coh[e, :], c_endog[e, :] + a_grid, a_grid)

27 µs ± 276 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


What about the new method?

In [31]:
interpolate_monotonic_loop(coh, c_endog + a_grid, a_grid) # burn-in
%timeit interpolate_monotonic_loop(coh, c_endog + a_grid, a_grid)

9.51 µs ± 46.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


A big improvement, by almost a factor of three!

**One minor improvement: enforcing the borrowing constraint better.** The line profiling of `backward_iteration` above does not reveal many other places where much improvement is possible. The only line that slightly sticks out is line 99, where the borrowing constraint is enforced by writing `a = np.maximum(a, a_grid[0])`.

This is an inefficient way to update `a` for two reasons. First, it creates an entirely new array in memory, rather than just altering `a` in place. Second, it compares every single entry of `a` to `a_grid[0]`, when the only entries of `a[e, :]` that are likely to violate the borrowing constraint, for each `e`, are a few early entries. Once we find one that is above `a_grid[0]`, the monotonicity of `a` means that we can stop looking.

Let's write a simple Numba-accelerated function.

In [32]:
@numba.njit
def setmin(x, xmin):
    """Set 2-dimensional array x, where each row is ascending, equal to max(x, xmin)."""
    ni, nj = x.shape
    for i in range(ni):
        for j in range(nj):
            if x[i, j] < xmin:
                # if below minimum, enforce minimum
                x[i, j] = xmin
            else:
                # otherwise, do nothing, and skip to next row (thanks to monotonicity)
                break

**Putting it all together.** We'll now write a new `backward_iteration` function that replaces our previous interpolation and constraint enforcement with the new functions:

In [33]:
def backward_iteration(Va, Pi, a_grid, y, r, beta, eis):
    # step 1: discounting and expectations
    Wa = beta * Pi @ Va
    
    # step 2: solving for asset policy using the first-order condition
    c_endog = Wa**(-eis)
    coh = y[:, np.newaxis] + (1+r)*a_grid
    a = interpolate_monotonic_loop(coh, c_endog + a_grid, a_grid)
        
    # step 3: enforcing the borrowing constraint and backing out consumption
    setmin(a, a_grid[0])
    c = coh - a
    
    # step 4: using the envelope condition to recover the derivative of the value function
    Va = (1+r) * c**(-1/eis)
    
    return Va, a, c

Let's time the original version:

In [34]:
backward_inputs = ss['Va'], ss['Pi'], ss['a_grid'], ss['y'], ss['r'], ss['beta'], ss['eis']
%timeit sim.backward_iteration(*backward_inputs)

38.1 µs ± 280 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


and now the new version:

In [35]:
backward_iteration(*backward_inputs) # burn-in
%timeit backward_iteration(*backward_inputs)

20.7 µs ± 414 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


We've sped this up by a factor of slightly less than 2. Now let's place this as part of a new `policy_ss` function, where we also eliminate most of the cost of checking for convergence through the same techniques as in the last section:

In [36]:
def policy_ss(Pi, a_grid, y, r, beta, eis, tol=1E-9):
    # initial guess for Va: assume consumption 5% of cash-on-hand, then get Va from envelope condition
    coh = y[:, np.newaxis] + (1+r)*a_grid
    c = 0.05 * coh
    Va = (1+r) * c**(-1/eis)
    
    # iterate until maximum distance between two iterations falls below tol, fail-safe max of 10,000 iterations
    for it in range(10_000):
        Va, a, c = backward_iteration(Va, Pi, a_grid, y, r, beta, eis)
        
        # after iteration 0, can compare new policy function to old one
        if it % 10 == 1 and equal_tolerance(a, a_old, tol):
            return Va, a, c
        
        a_old = a

Time the old `policy_ss`:

In [37]:
%timeit sim.policy_ss(**calibration)

23 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


and now the new one:

In [38]:
policy_ss(**calibration) # burn-in (just in case)
%timeit policy_ss(**calibration)

11.1 ms ± 70.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We've achieved a speed-up of slightly more than 2x.

<a id="section4"></a>
# 4 Overall steady-state performance
Timing the old steady-state function:

In [39]:
%timeit sim.steady_state(**calibration)

31.8 ms ± 86.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


and now writing a new one that calls upon the functions we've defined in this notebook:

In [40]:
def steady_state(Pi, a_grid, y, r, beta, eis):
    Va, a, c = policy_ss(Pi, a_grid, y, r, beta, eis)
    D = distribution_ss(Pi, a, a_grid)
    
    return dict(D=D, Va=Va, 
                a=a, c=c,
                A=np.vdot(a, D), C=np.vdot(c, D),
                Pi=Pi, a_grid=a_grid, y=y, r=r, beta=beta, eis=eis)

In [41]:
steady_state(**calibration) # burn-in (just in case)
%timeit steady_state(**calibration)

17 ms ± 88.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The overall improvement in performance is a factor of almost 2. At 17 ms on an ordinary laptop, the time to calculate a steady state for a model with a $7\times 500=3500$-dimensional steady state is on par with the current state of the art.

<a id="section5"></a>
# 5 Addendum: replacing `get_lottery`
The current `get_lottery` function converts the asset policy, which generally takes a value between gridpoints, to a lottery over gridpoints.

As we pointed out after the line profiling in section 2, this does not impose much of a cost in the steady state since it only needs to be done once, but in dynamic applications it will need to be applied every period, and the current `get_lottery` is surprisingly costly for that purpose.

Fortunately, the `interpolate_monotonic` function that we defined above does, as an intermediate step, exactly what we want for `get_lottery`. We'll copy most of `interpolate_monotonic` below, and modify it so that the $i$ and $\pi$ are reported.

In [42]:
@numba.njit
def interpolate_lottery(x, xp):
    """Given a grid of xp, for each entry x_cur in (increasing) x, find the i and pi
    such that x_cur = pi*xp[i] + (1-pi)*xp[i+1], where xp[i] and xp[i+1] bracket x_cur"""
    nx, nxp = x.shape[0], xp.shape[0]
    i = np.empty(nx, dtype=np.int64)
    pi = np.empty(nx)
    
    # at any given moment, we are looking between data points with indices xp_i and xp_i+1
    # we'll keep track of xp_i and also the values of xp at xp_i and xp_i+1
    # (note: if x is outside range of xp, we'll use closest data points and extrapolate)
    xp_i = 0
    xp_lo = xp[xp_i]
    xp_hi = xp[xp_i + 1]
    
    # iterate through all points in x
    for xi_cur in range(nx):
        x_cur = x[xi_cur]
        while xp_i < nxp - 2:
            # if current x (x_cur) is below upper data point (xp_hi), we're good
            if x_cur < xp_hi:
                break
                
            # otherwise, we need to look at the next pair obf data points until x_cur is less than xp_hi
            # (so between xp_lo and xp_hi)
            xp_i += 1
            xp_lo = xp_hi
            xp_hi = xp[xp_i + 1]

        # find the pi such that x_cur = pi*x_lo + (1-pi)*x_hi
        i[xi_cur] = xp_i
        pi[xi_cur] = (xp_hi - x_cur) / (xp_hi - xp_lo)
    return i, pi

and as we did before with `interpolate_monotonic`, we'll make a loop around this:

In [43]:
@numba.njit
def interpolate_lottery_loop(x, xp):
    i = np.empty_like(x, dtype=np.int64)
    pi = np.empty_like(x)
    for e in range(x.shape[0]):
        i[e, :], pi[e, :] = interpolate_lottery(x[e, :], xp)
    return i, pi

Let's make sure that this gives the same answer as `get_lottery` from before—in this case exactly the same, without even small numerical differences:

In [44]:
i, pi = interpolate_lottery_loop(a, a_grid)
i2, pi2 = sim.get_lottery(a, a_grid)
np.all(i == i2), np.all(pi == pi2)

(True, True)

What about the relative speeds? We've cut by a factor of around 20, which will be very useful when we need to apply this repeatedly in dynamic applications:

In [45]:
%timeit interpolate_lottery_loop(a, a_grid)
%timeit sim.get_lottery(a, a_grid)

8.49 µs ± 52 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
203 µs ± 3.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
